In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 16.6MB/s 
     |████████████████████████████████| 901kB 40.7MB/s 
     |████████████████████████████████| 3.3MB 49.5MB/s 


In [2]:
!pip install pdfminer

     |████████████████████████████████| 4.2MB 20.3MB/s 
     |████████████████████████████████| 1.9MB 56.2MB/s 
  Created wheel for pdfminer: filename=pdfminer-20191125-cp37-none-any.whl size=6140091 sha256=fa08eb72973fcd041abb81c8a0f16f7dcc8c904293b680dd3ae6d4c674fe8d8c
  Stored in directory: /root/.cache/pip/wheels/e1/00/af/720a55d74ba3615bb4709a3ded6dd71dc5370a586a0ff6f326
Successfully built pdfminer


In [3]:
import re

In [5]:
from io import StringIO
import urllib.request
import pandas as pd
import time

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
# %load_ext google.colab.data_table

def pdf_to_text(filename):
  output_string = StringIO()
  with open(filename, 'rb') as in_file:
      parser = PDFParser(in_file)
      doc = PDFDocument(parser)
      rsrcmgr = PDFResourceManager()
      device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
      interpreter = PDFPageInterpreter(rsrcmgr, device)
      for page in PDFPage.create_pages(doc):
          interpreter.process_page(page)
  return output_string.getvalue()

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [31]:
def preprocess_text(filename):
  text = pdf_to_text(filename)
  text = text.partition('Introduction')[2]
  if 'Acknowledgements' in text:
    text = text.partition('Acknowledgements')[0].strip()
  else:
    text = text.partition('References')[0].strip()
  # print(text)
  text = re.sub('ﬀ','ff',text)
  text = re.sub('[(].+[)]','',text)
  text = re.sub('[[][^a-zA-Z]+[]]','',text)
  text = re.sub('-\n','',text)
  text = re.sub('\n',' ',text)
  paragraphs = re.split('\s{2,}',text)
  """for p in paragraphs:
    print(p+'\n')"""
  new_paragraphs = [p for p in paragraphs if re.search('[a-z]+',p,re.I)]

  paragraphs = []
  new_p = ""
  for p in new_paragraphs:
    if len(new_p) != 0:
      new_p += ' '+p
      if p.endswith('.') and re.search('al\s*.$',p) == None:
        paragraphs.append(new_p)
        new_p = ""
    else:
      if p.endswith('.') and re.search('al\s*.$',p) == None:
        paragraphs.append(p)
      else:
        new_p = p

  new_paragraphs = [p for p in paragraphs if re.search('Table.+:',p) == None and re.search('Fig.+:',p) == None]

  return new_paragraphs

The goal of object detection is to predict a set of bounding boxes and category labels for each object of interest. Modern detectors address this set prediction task in an indirect way, by deﬁning surrogate regression and classiﬁcation problems on a large set of proposals , anchors , or window centers . Their performances are signiﬁcantly inﬂuenced by postprocessing steps to collapse near-duplicate predictions, by the design of the anchor sets and by the heuristics that assign target boxes to anchors . To simplify these pipelines, we propose a direct set prediction approach to bypass the surrogate tasks. This end-to-end philosophy has led to signiﬁcant advances in complex structured prediction tasks such as machine translation or speech recognition, but not yet in object detection: previous attempts either add other forms of prior knowledge, or have not proven to be competitive with strong baselines on challenging benchmarks. This paper aims to bridge this gap.

We streamline the train

In [35]:
from transformers import pipeline, BartTokenizer

# max seq length for this model = 1024
summarizer = pipeline(task="summarization", model="facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [38]:
def generate_chunks(new_paragraphs):
  large_paragraph = ""
  modified_paragraphs = []
  for paragraph in new_paragraphs:
    if len(tokenizer.encode(paragraph)) > 1024:
      sents = nltk.sent_tokenize(paragraph)
      small_paragraph = ""
      paragraphs = []
      for sent in sents:
        if len(tokenizer.encode(small_paragraph))+len(tokenizer.encode(sent)) < 1024:
          small_paragraph += ' '+sent
        else:
          paragraphs.append(small_paragraph)
          small_paragraph = ""
      modified_paragraphs.extend(paragraphs)
    else:
      if len(tokenizer.encode(large_paragraph)) + len(tokenizer.encode(paragraph))+1 < 1024:
        large_paragraph += ' '+paragraph
      else:
        modified_paragraphs.append(large_paragraph)
        # print(len(large_paragraph))
        large_paragraph = paragraph

In [39]:
for p in modified_paragraphs:
  print(len(p))

4128
4959
3508
3641
3392
4491
3643


In [40]:
def generate_summarization(modified_paragpraphs):
  summarization = []

  for paragraph in modified_paragraphs:
      summarized = summarizer(paragraph)
      summarization.append(summarized[0]['summary_text'])

  return '\n'.join(summarization)

 object detection is a direct set prediction problem. We adopt an encoder-decoder architecture based on transformers. We train end-to-end with a set loss function which performs bipartite matching between predicted and ground-truth objects. We evaluate DETR on one of the most popular object detection datasets, COCO.
There is no canonical deep learning model to directly predict sets. We follow the bipartite matching loss approach. We combine transformers and parallel decoding for their suitable trade-off between computational cost and the ability to perform the global computations required for set prediction. We describe our architecture in detail in Figure 2.
DETR infers a ﬁxed-size set of N predictions, in a single pass through the decoder. The loss produces an optimal bipartite matching between predicted and ground truth objects. This optimal assignment is computed eﬃciently with the Hungarian algorithm. The matching cost takes into account both the class prediction and the similarit

In [ ]:
paragraphs = preprocess_text('filename')
chunks = generate_chunks(paragraphs)
summary = generate_summarization(chunks)
print(summary)